In [1]:
#CLUSTER GORDIAS LIS 
import pyTigerGraph as tg
import pandas as pd
import gradio as gr
graph = tg.TigerGraphConnection(host=hostName, graphname=graphName)

authToken = graph.getToken(secret)
authToken = authToken[0]
print(f"secret token: {authToken}")
conn = tg.TigerGraphConnection(host=hostName, graphname=graphName, username=userName, password=password, apiToken=authToken)

c:\Users\JulienRigot\OneDrive - LIS Data Solutions\Escritorio\code_GORDIAS\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ConnectionError: HTTPSConnectionPool(host='0eede95b6ddc4300bdc7b3103098e51c.i.tgcloud.io', port=9000): Max retries exceeded with url: /requesttoken (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000001D86CB303E0>: Failed to resolve '0eede95b6ddc4300bdc7b3103098e51c.i.tgcloud.io' ([Errno 11001] getaddrinfo failed)"))

In [2]:
conn.getSchema()

{'GraphName': 'VWG',
 'VertexTypes': [{'Config': {'STATS': 'OUTDEGREE_BY_EDGETYPE'},
   'Attributes': [{'AttributeType': {'Name': 'FLOAT'},
     'AttributeName': 'LoadCapacity'},
    {'AttributeType': {'Name': 'FLOAT'}, 'AttributeName': 'UnloadCapacity'},
    {'AttributeType': {'Name': 'FLOAT'}, 'AttributeName': 'Stock'},
    {'AttributeType': {'Name': 'STRING'}, 'AttributeName': 'Carga'},
    {'AttributeType': {'Name': 'FLOAT'}, 'AttributeName': 'Capacity'},
    {'AttributeType': {'Name': 'FLOAT'}, 'AttributeName': 'latitude'},
    {'AttributeType': {'Name': 'FLOAT'}, 'AttributeName': 'longitude'}],
   'PrimaryId': {'AttributeType': {'Name': 'STRING'}, 'AttributeName': 'id'},
   'Name': 'Nodes'}],
 'EdgeTypes': [{'IsDirected': False,
   'ToVertexTypeName': 'Nodes',
   'Config': {},
   'Attributes': [{'AttributeType': {'Name': 'FLOAT'},
     'AttributeName': 'Price'},
    {'AttributeType': {'Name': 'STRING'}, 'AttributeName': 'Carga'},
    {'AttributeType': {'Name': 'FLOAT'}, 'Attribut

In [7]:
result = conn.gsql("""
USE GRAPH transport_demo
CREATE QUERY tg_astar (VERTEX source_vertex, VERTEX target_vertex, SET<STRING> e_type_set,
STRING weight_type, STRING latitude, STRING longitude,
STRING weight_attribute, BOOL print_stats = False) SYNTAX V1 {

TYPEDEF TUPLE<FLOAT dist, VERTEX v> pathTuple;    # <shotest distance, parent node>
HeapAccum<pathTuple>(1, dist ASC) @@find_min_v_heap;  # retain 1 shortest path
HeapAccum<pathTuple>(1, dist ASC) @min_dist_heap;
OrAccum @or_visited, @@or_valid_path_exists;
ListAccum<VERTEX> @@tmp_list;  # the optimal node
SumAccum<FLOAT> @@sum_total_dist;  # the shortest distance
SetAccum<EDGE> @@display_edge_set;
SetAccum<VERTEX> @@display_node_set;
INT hop;
FLOAT x1,y1;

# Check weight_type parameter
IF weight_type NOT IN ("INT", "FLOAT", "DOUBLE") THEN
    PRINT "weight_type must be INT, FLOAT, or DOUBLE" AS errMsg;
    RETURN;
END;

# record target latitude and longitude
Tgt = {target_vertex};
Tgt = SELECT s
      FROM Tgt:s
      POST-ACCUM x1 = s.getAttr(latitude,"FLOAT"),
	         y1 = s.getAttr(longitude,"FLOAT");

Start = {source_vertex};   # the optimal node
Opt = {source_vertex};    # all of the optimal nodes

Start = SELECT s
        FROM Start:s
        ACCUM s.@or_visited = True,
              s.@min_dist_heap = pathTuple(0,s);

# run aster to find shortest distance greedily
WHILE Opt.size() > 0 DO
    # find the node with shortest distance
    Opt = SELECT t
          FROM Opt:s-(e_type_set:e)-> :t
          WHERE t.@or_visited == False
	  ACCUM
              # we use Haversine formula as the heuristic function here
              CASE weight_type WHEN "INT" THEN
                  t.@min_dist_heap += pathTuple(s.@min_dist_heap.top().dist + e.getAttr(weight_attribute, "INT")
		  +  tg_GetDistance(t.getAttr(latitude,"FLOAT"),t.getAttr(longitude,"FLOAT"),x1,y1),s)
              WHEN "FLOAT" THEN
                  t.@min_dist_heap += pathTuple(s.@min_dist_heap.top().dist + e.getAttr(weight_attribute, "FLOAT")
		  +  tg_GetDistance(t.getAttr(latitude,"FLOAT"),t.getAttr(longitude,"FLOAT"),x1,y1),s)
              WHEN "DOUBLE" THEN
                  t.@min_dist_heap += pathTuple(s.@min_dist_heap.top().dist + e.getAttr(weight_attribute, "DOUBLE")
		  +  tg_GetDistance(t.getAttr(latitude,"FLOAT"),t.getAttr(longitude,"FLOAT"),x1,y1),s)
              END;

    Opt = SELECT t
          FROM Start:s-(e_type_set:e)-> :t
          WHERE t.@or_visited == False
          POST-ACCUM @@find_min_v_heap += pathTuple(t.@min_dist_heap.top().dist,t);

    @@tmp_list.clear();
    IF @@find_min_v_heap.size() > 0 THEN
        @@tmp_list += @@find_min_v_heap.pop().v;
    END;

    Opt = {@@tmp_list};
    Start = Opt UNION Start;
    Opt = SELECT t
          FROM Opt:t
	  POST-ACCUM
	      t.@or_visited += True;
              # Determine if it is the target point and terminate the loop if it is
              IF @@tmp_list.get(0) == target_vertex THEN
                  BREAK;
              END;

END;
# The test is whether there is a path between two points
Start = {target_vertex};
Start = SELECT s
        FROM Start:s
        POST-ACCUM @@or_valid_path_exists += s.@min_dist_heap.size() > 0,
                   @@display_node_set += s;

IF @@or_valid_path_exists THEN
    # find path
    WHILE Start.size() > 0 DO
        Start =
	SELECT t
	FROM Start:s-(e_type_set:e)-> :t
        WHERE t == s.@min_dist_heap.top().v
        ACCUM
	    @@display_edge_set += e,
            CASE weight_type WHEN "INT" THEN
                @@sum_total_dist += e.getAttr(weight_attribute, "INT")
            WHEN "FLOAT" THEN
                @@sum_total_dist += e.getAttr(weight_attribute, "FLOAT")
            WHEN "DOUBLE" THEN
                @@sum_total_dist += e.getAttr(weight_attribute, "DOUBLE")
            END,
            @@display_node_set += t;
            hop = hop + 1;
    END;
    hop = hop - 1;
    PRINT @@sum_total_dist;
    PRINT hop;
    IF print_stats THEN
        tmp = {@@display_node_set};
        PRINT @@display_edge_set,tmp;
    END;
ELSE
    PRINT "No viable path found.";
END;
}""")

HTTPError: 401 Client Error: Unauthorized for url: https://director-lisdatasolutions.i.tgcloud.io:443/gsqlserver/gsql/file

In [17]:
def erase_edge(conn, source_vertex_id, target_vertex_id, vertex_type="Nodes", edgeType="distribute_to"):
    global logs_modif_network
    
    try:
        result = conn.delEdges(vertex_type,source_vertex_id,edgeType,vertex_type,target_vertex_id)
        
        # Afficher le résultat pour comprendre sa structure
        print("Result of delEdges:", result)
        
        # Si le format attendu est différent, on peut adapter le traitement ici
        if isinstance(result, list):  # Si le résultat est une liste
            for r in result:
                print(f"Deleted edges: {r.get('deleted_edges')}, Edge Type: {r.get('e_type')}")
        elif isinstance(result, dict):  # Si c'est un dictionnaire
            print(f"Deleted edges: {result.get('deleted_edges')}, Edge Type: {result.get('e_type')}")
        else:
            print("Unexpected result format")
        
        return result
    
    except TypeError as e:
        print(f"Error: {e}")
        raise


In [18]:
erase_edge(conn,"Madrid","Seville")

Error: string indices must be integers, not 'str'


TypeError: string indices must be integers, not 'str'

In [19]:
conn.getSchema()

{'GraphName': 'VWG',
 'VertexTypes': [{'Config': {'STATS': 'OUTDEGREE_BY_EDGETYPE'},
   'Attributes': [{'AttributeType': {'Name': 'FLOAT'},
     'AttributeName': 'LoadCapacity'},
    {'AttributeType': {'Name': 'FLOAT'}, 'AttributeName': 'UnloadCapacity'},
    {'AttributeType': {'Name': 'FLOAT'}, 'AttributeName': 'Stock'},
    {'AttributeType': {'Name': 'STRING'}, 'AttributeName': 'Carga'},
    {'AttributeType': {'Name': 'FLOAT'}, 'AttributeName': 'Capacity'},
    {'AttributeType': {'Name': 'FLOAT'}, 'AttributeName': 'latitude'},
    {'AttributeType': {'Name': 'FLOAT'}, 'AttributeName': 'longitude'}],
   'PrimaryId': {'AttributeType': {'Name': 'STRING'}, 'AttributeName': 'id'},
   'Name': 'Nodes'}],
 'EdgeTypes': [{'IsDirected': False,
   'ToVertexTypeName': 'Nodes',
   'Config': {},
   'Attributes': [{'AttributeType': {'Name': 'FLOAT'},
     'AttributeName': 'Price'},
    {'AttributeType': {'Name': 'STRING'}, 'AttributeName': 'Carga'},
    {'AttributeType': {'Name': 'FLOAT'}, 'Attribut

In [ ]:
# # FREE CLUSTER EDGE
hostName1 = "https://506118e7a5824411b8cff3b579d8809b.i.tgcloud.io"
graphName1 = "VWG1"
secret1 = "4alh4gsrhiq5tlfb3tf5gro5srpa5e35"
userName1 = "user_1"
password1 = "A1z2e3r4*"
tigergraph_insights_map1 = "https://tools.tgcloud.io/insights/app/gBQu5GpNcMMoNttUGH8t29/page/mwMTnWnUDwp1PgFEE5Vapi/widgetShare/veFZVtxZW1gpd4GXchssZ2?domain=506118e7a5824411b8cff3b579d8809b.i&orgName=jrlisdata-org-2024325&clusterid=95a180a6-6c60-4d83-94ff-5291bf665469&TigerGraphToken=2cf416dd-3e79-4428-a5c0-1a68168b608b"
graph1 = tg.TigerGraphConnection(host=hostName1, graphname=graphName1)

authToken1 = graph1.getToken(secret1)
authToken1 = authToken1[0]
print(f"secret token 1: {authToken1}")
conn1 = tg.TigerGraphConnection(host=hostName1, graphname=graphName1, username=userName1, password=password1, apiToken=authToken1)

secret token 1: 395kqmln948q1cf8aspr7vnd479l216i


# Funciones para las entregas

In [4]:
from concurrent.futures import ThreadPoolExecutor, as_completed
import threading
atributes_nodes = {}
atributes_edges = {}


def path_taken(results):

    """
    Determina y devuelve el orden de los nodos que conforman el camino tomado a partir de los resultados
    de una consulta de ruta en el grafo (el algoritmo suele devolver los resultado en el desorden)
    """

    # Extrae el conjunto de aristas desde el diccionario de resultados.
    edges = results[2]['@@display_edge_set']
    
    # Inicializa una variable para almacenar el nodo inicial.
    node_initial = None
    
    # Crea un conjunto de identificadores de nodos a los que se dirige cada arista.
    edge_to_ids = {edge['to_id'] for edge in edges}
    
    # Busca el nodo inicial (el nodo que no es el destino de ningún arista).
    for edge in edges:
        if edge['from_id'] not in edge_to_ids:
            node_initial = edge['from_id']
            break
    
    # Si no se encuentra un nodo inicial, lanza una excepción.
    if node_initial is None:
        raise ValueError("initial node cannot be determined.")
    
    # Inicializa el nodo actual como el nodo inicial y crea una lista para el camino.
    current_node = node_initial
    path = [current_node]
    
    # Recorre los bordes para construir el camino desde el nodo inicial.
    while True:
        next_node = None
        
        # Busca el siguiente nodo en el camino basado en el nodo actual.
        for edge in edges:
            if edge['from_id'] == current_node:
                next_node = edge['to_id']
                break
        
        # Si no se encuentra el siguiente nodo, termina el bucle.
        if next_node is None:
            break
        
        # Añade el siguiente nodo al camino y actualiza el nodo actual.
        path.append(next_node)
        current_node = next_node
    
    # Crea un diccionario de nodos mapeando cada nodo a sí mismo.
    nodes = {node_name: node_name for node_name in path}
    
    # Crea una lista ordenada de nodos basada en el camino encontrado.
    ordered_nodes = [nodes[node_name] for node_name in path]
    
    # Devuelve la lista de nodos ordenados que representa el camino tomado.
    return ordered_nodes


def add_or_update_final_nodes_batch(nodes_batch_full, nodes_batch, charge):

    """
    Actualiza o agrega nodos en el lote completo de nodos basado en las transferencias realizadas, ajustando el stock según la carga.
    """

    # Crea una lista de ciudades existentes a partir de los nodos en 'nodes_batch_full'.
    existing_cities = [city for city, _ in nodes_batch_full]
    
    # Itera sobre cada nodo en el 'nodes_batch' para actualizar o agregar nodos en 'nodes_batch_full'.
    for node, _ in nodes_batch:
        if node in existing_cities:
            # Si el nodo ya existe en 'nodes_batch_full', busca el índice del nodo para actualizar el stock.
            for i, (city, stock) in enumerate(nodes_batch_full):
                if city == node:
                    # Si es el primer nodo, decrementa el stock por 'charge'.
                    # De lo contrario, incrementa el stock por 'charge'.
                    if i == 0:
                        nodes_batch_full[i] = (city, {"Stock": nodes_batch_full[i][1]["Stock"] - charge})
                    else:
                        nodes_batch_full[i] = (city, {"Stock": nodes_batch_full[i][1]["Stock"] + charge})
            continue
        else:
            # Si el nodo no está en 'nodes_batch_full', agrega nuevas entradas para los nodos de 'nodes_batch'.
            nodes_batch_full.extend([
                (nodes_batch[0][0], {"Stock": nodes_batch[0][1]["Stock"]}),
                (nodes_batch[1][0], {"Stock": nodes_batch[1][1]["Stock"]})
            ])
            # Retorna 'nodes_batch_full' después de agregar las nuevas entradas.
            return nodes_batch_full
    
    # Retorna 'nodes_batch_full' después de actualizar o agregar los nodos.
    return nodes_batch_full

def add_or_update_final_edges_batch(edges_batch_full, edges_batch, charge):

    """
    Actualiza o agrega aristas en el lote completo de aristas basado en las transferencias realizadas, ajustando el movimiento diario según la carga.
    """


    # Crea una lista de bordes existentes a partir de 'edges_batch_full', considerando ambos sentidos.
    existing_edges = [(edge[0], edge[1]) for edge in edges_batch_full]
    
    # Itera sobre cada borde en 'edges_batch' para actualizar o agregar bordes en 'edges_batch_full'.
    for edge in edges_batch:
        # Verifica si el borde ya existe en 'edges_batch_full' considerando ambos sentidos.
        if (edge[0], edge[1]) in existing_edges or (edge[1], edge[0]) in existing_edges:
            # Si el borde ya existe, busca el índice del borde para actualizar el movimiento diario.
            for i, (origin, destination, movement) in enumerate(edges_batch_full):
                # Compara ambos sentidos del borde para encontrar el índice correcto.
                if (origin, destination) == (edge[0], edge[1]) or (origin, destination) == (edge[1], edge[0]):
                    # Actualiza el movimiento diario del borde sumando 'charge'.
                    edges_batch_full[i] = (origin, destination, {"Daily_movement": edges_batch_full[i][2]["Daily_movement"] + charge})
            continue
        else:
            # Si el borde no existe, agrega una nueva entrada en 'edges_batch_full'.
            edges_batch_full.append((edge[0], edge[1], {"Daily_movement": edge[2]["Daily_movement"]}))
    
    # Retorna 'edges_batch_full' después de actualizar o agregar los bordes.
    return edges_batch_full


def actualize_graph_carga(conn=conn):
    """
    Actualiza el estado de carga de los nodos y aristas en el grafo, clasifica cada uno según su nivel de carga y registra los estados detectados
    Esto se hace con el objetivo de camibar los colores en funccion de la carga en el mapa de Tigergraph, creo que no es necesario para prod.
    """

    # Obtiene la lista de nodos y aristas desde la conexión proporcionada.
    nodes = conn.getVertices("Nodes")
    aristas = conn.getEdgesByType(edgeType="distribute_to")

    # Inicializa listas para almacenar nodos y aristas con datos escalados.
    nodes_batch_scale = []
    aristas_batch_scale = []
    
    # Inicializa una variable para registrar los estados de nodos y aristas.
    logs_state_nodes_and_arristas = ""

    # Contadores para los diferentes niveles de carga de nodos y aristas.
    node_counts = {"Sobrecargado": 0, "Subcargado": 0, "Optimal": 0}
    arista_counts = {"Sobrecargado": 0, "Subcargado": 0, "Optimal": 0}

    # Itera sobre cada nodo para determinar su nivel de carga.
    for node in nodes:
        node_capacidad = node["attributes"]["Capacity"]
        node_stock = node["attributes"]["Stock"]
        
        # Determina el nivel de carga del nodo basado en su capacidad y stock.
        if node_capacidad == 0:
            nivel_stock = "No definido"
        else:
            if (node_stock / node_capacidad) > 1:
                nivel_stock = "Sobrecargado"
                node_counts["Sobrecargado"] += 1
                logs_state_nodes_and_arristas += f"⚠️ {node['v_id']} es {nivel_stock} \n"
            elif 0.7 < (node_stock / node_capacidad) <= 1:
                nivel_stock = "Optimal"
                node_counts["Optimal"] += 1
            else:
                nivel_stock = "Subcargado"
                logs_state_nodes_and_arristas += f"⚠️ {node['v_id']} es {nivel_stock} \n"
                node_counts["Subcargado"] += 1

        # Añade el nodo con su nivel de carga a la lista de nodos a escalar.
        nodes_batch_scale.append((node["v_id"], {"Carga": nivel_stock}))

    # Itera sobre cada arista para determinar su nivel de carga.
    for arista in aristas:
        capacity_edge = arista["attributes"]["Capacity"]
        daily_movement = arista["attributes"]["Daily_movement"]
        
        # Determina el nivel de carga de la arista basado en su capacidad y movimiento diario.
        if capacity_edge == 0:
            nivel_arista = "No definido"
        else:
            if (daily_movement / capacity_edge) > 1:
                nivel_arista = "Sobrecargado"
                arista_counts["Sobrecargado"] += 1
                logs_state_nodes_and_arristas += f"⚠️ la arista {arista['from_id']} hasta {arista['to_id']} es {nivel_arista} \n"
            elif 0.7 < (daily_movement / capacity_edge) <= 1:
                nivel_arista = "Optimal"
                arista_counts["Optimal"] += 1
            else:
                nivel_arista = "Subcargado"
                logs_state_nodes_and_arristas += f"⚠️ la arista {arista['from_id']} hasta {arista['to_id']} es {nivel_arista} \n"
                arista_counts["Subcargado"] += 1

        # Añade la arista con su nivel de carga a la lista de aristas a escalar.
        aristas_batch_scale.append((arista["from_id"], arista["to_id"], {"Carga": nivel_arista}))

    # Actualiza los nodos en la base de datos con los datos escalados.
    conn.upsertVertices("Nodes", nodes_batch_scale)
    
    # Actualiza las aristas en la base de datos con los datos escalados.
    conn.upsertEdges(sourceVertexType="Nodes", targetVertexType="Nodes", edgeType="distribute_to", edges=aristas_batch_scale)

    # Si no se registraron cambios, indica que todos los nodos y aristas están en un estado óptimo.
    if logs_state_nodes_and_arristas == "":
        logs_state_nodes_and_arristas = "Todos los nodos y aristas están en un estado óptimo."

    # Crea un diccionario con los registros de estado y los conteos de nodos y aristas.
    results = {
        "logs": logs_state_nodes_and_arristas,
        "node_counts": node_counts,
        "arista_counts": arista_counts
    }

    # Retorna el diccionario con los resultados.
    return results


# Main transfer function

def transfert_nodes_and_edges(node_initial, node_final, charge, weight_attribute="Capacity", nodetype="Nodes", edgetype="distribute_to"):
    
    """
    Realiza la transferencia de una carga desde un nodo inicial a un nodo final siguiendo el camino óptimo encontrado por el algoritmo A*
    actualizando los stocks de los nodos y el movimiento diario de las aristas involucradas.
    """

    # Lanza la función A* para encontrar el camino óptimo entre node_initial y node_final
    results_astar = conn.runInstalledQuery("tg_astar", params={
        "source_vertex": node_final, "source_vertex.type": nodetype,
        "target_vertex": node_initial, "target_vertex.type": nodetype,
        "e_type_set": edgetype, "weight_type": "FLOAT",
        "latitude": "latitude", "longitude": "longitude",
        "weight_attribute": weight_attribute,
        "print_stats": "True"
    })
    
    # Obtiene el orden de los nodos en el camino encontrado
    order_taken = path_taken(results_astar)
    
    # Obtiene el conjunto de aristas involucradas en el camino
    edge_set = results_astar[2]['@@display_edge_set']
    
    # Reorganiza las aristas de acuerdo al orden de los nodos en el camino
    reordered_edges_set = []
    for location in order_taken:
        for edge in edge_set:
            if edge['from_id'] == location:
                reordered_edges_set.append(edge)

    # Obtiene el conjunto de nodos involucrados en el camino
    node_set = results_astar[2]['tmp']
    
    # Reorganiza los nodos de acuerdo al orden en el camino
    reordered_nodes_set = []
    for location in order_taken:
        for item in node_set:
            if item['v_id'] == location:
                reordered_nodes_set.append(item)

    try:
        # Recupera los atributos de los nodos inicial y final
        node_i_stock = reordered_nodes_set[0]["attributes"]["Stock"]
        node_f_stock = reordered_nodes_set[-1]["attributes"]["Stock"]
        node_i_unload_capacity = reordered_nodes_set[0]["attributes"]["UnloadCapacity"]
        node_f_load_capacity = reordered_nodes_set[-1]["attributes"]["LoadCapacity"]

        # Verifica si hay suficiente inventario en el nodo inicial
        if charge > node_i_stock:
            error_message = f"{node_initial} hasta {node_final}: No hay suficiente inventario ({charge} > {node_i_stock})\n"
            log_list += error_message

        # Verifica si hay suficiente capacidad de carga en el nodo final
        if charge > node_f_load_capacity:
            error_message = f"{node_initial} hasta {node_final}: No hay suficiente capacidad de carga en el almacén de recepción (charge: {charge} > LoadCapacity: {node_f_load_capacity})\n"
            log_list += error_message

        # Verifica si hay suficiente capacidad de descarga en el nodo inicial
        if charge > node_i_unload_capacity:
            error_message = f"{node_initial} hasta {node_final}: No hay suficiente capacidad de descarga desde el almacén de salida (charge: {charge} > UnloadCapacity: {node_i_unload_capacity})\n"
            log_list += error_message

        # Recupera la capacidad de la arista y el movimiento diario actual
        capacity_edge = reordered_edges_set[0]["attributes"]["Capacity"]
        daily_movement = reordered_edges_set[0]["attributes"]["Daily_movement"]

        # Verifica si la arista puede soportar el movimiento adicional de carga
        if daily_movement + charge > capacity_edge:
            error_message = f"Enviar {charge} palets desde {node_initial} hasta {node_final}: Capacidad de la arista insuficiente ({daily_movement + charge} > {capacity_edge})\n"
            log_list += error_message

        # Actualiza los inventarios de los nodos inicial y final
        nodes_batch = [
            (node_initial, {"Stock": node_i_stock - charge}),
            (node_final, {"Stock": node_f_stock + charge})
        ]

        # Actualiza el movimiento diario en las aristas recorridas y acumula el costo
        edges_batch = []
        for element in reordered_edges_set:
            element['attributes']['Daily_movement'] += charge
            edges_batch.append((element["from_id"], element["to_id"], {"Daily_movement": element["attributes"]["Daily_movement"]}))
            palets_cost = 5
            palets_cost += element["attributes"]["Price"]

        # Añade los nodos inicial y final a los conjuntos de nodos únicos
        unique_origin_nodes.add(node_initial)
        unique_destination_nodes.add(node_final)
        
        # Incrementa el conteo de operaciones y el número total de palets
        operation_count = 100
        operation_count += 1
        palets_number = 22
        palets_number += charge

        return nodes_batch , edges_batch

    # Manejo de excepciones en caso de que ocurra algún error durante el proceso
    except Exception as e:
        log_list += f"Error durante la transferencia de {node_initial} a {node_final}: {str(e)}\n"
        return [], []

    except Exception as e:
        error_message = f"Error en la transferancia: {e}\n"
        log_list += error_message
        return error_message


# Thread safety
lock = threading.Lock()


def make_daily_movements(conn=conn, input_file, column_origin, column_destination, column_transfert):

    """
    Funccion principal : actualiza los logs, hace las transferencias (entregas) en paralelo con threads, y actualiza el grafo
    """

    # Variables globales para rastrear estadísticas y almacenar datos.
    global operation_count, palets_number, palets_cost, unique_origin_nodes, unique_destination_nodes, log_list, nodes_batch_full, edges_batch_full, df
    operation_count = 0  # Conteo de operaciones realizadas.
    palets_number = 0  # Número total de palets transferidos.
    palets_cost = 0  # Costo total asociado a los palets transferidos.
    unique_origin_nodes = set()  # Conjunto de nodos de origen únicos.
    unique_destination_nodes = set()  # Conjunto de nodos de destino únicos.
    log_list = " "  # Registro de errores o mensajes importantes.
    nodes_batch_full = []  # Lista para almacenar nodos actualizados.
    edges_batch_full = []  # Lista para almacenar aristas actualizadas.

    # Lee el archivo CSV y carga los datos en un DataFrame.
    df = pd.read_csv(input_file)

    # Función para manejar la transferencia de volumen entre nodos.
    def perform_transfer(row):
        global log_list
        try:
            # Obtiene el origen, destino y volumen desde la fila del DataFrame.
            origin = str(row[column_origin])
            destination = str(row[column_destination])
            volume = float(row[column_transfert])
            
            # Llama a la función para obtener nodos y aristas actualizados.
            nodes_batch, edges_batch = transfert_nodes_and_edges(origin, destination, volume)
            
            # Usa un lock para garantizar que las actualizaciones a las listas sean seguras en un entorno multihilo.
            with lock:
                # Actualiza o agrega nodos y aristas a las listas correspondientes.
                add_or_update_final_nodes_batch(nodes_batch_full, nodes_batch, volume)
                add_or_update_final_edges_batch(edges_batch_full, edges_batch, volume)
            
            # Actualiza las métricas globales.
            global operation_count, palets_number, palets_cost
            operation_count += 1  # Incrementa el conteo de operaciones.
            palets_number += volume  # Incrementa el número total de palets.
            palets_cost += volume  # Incrementa el costo total (asumiendo un costo por palet definido como some_cost_per_palet).
            unique_origin_nodes.add(origin)  # Agrega el nodo de origen al conjunto de nodos únicos.
            unique_destination_nodes.add(destination)  # Agrega el nodo de destino al conjunto de nodos únicos.
        
        except Exception as e:
            # Registra un mensaje de error si ocurre una excepción.
            error_message = f"{origin} hasta {destination}, no camino posible.\n"
            log_list += error_message

    # Usa un ThreadPoolExecutor para procesar las transferencias en paralelo.
    with ThreadPoolExecutor(max_workers=1000) as executor:
        # Crea y envía tareas para cada fila del DataFrame.
        futures = [executor.submit(perform_transfer, row) for index, row in df.iterrows()]
        # Espera a que todas las tareas se completen.
        for future in as_completed(futures):
            pass  # Se pueden manejar los resultados de las tareas aquí si es necesario.

    # Actualiza los nodos y aristas en la base de datos.
    conn.upsertVertices("Nodes", nodes_batch_full)
    nodetype = "Nodes"
    edgetype = "distribute_to"
    conn.upsertEdges(sourceVertexType=nodetype, targetVertexType=nodetype, edgeType=edgetype, edges=edges_batch_full)
    
    # Llama a la función para actualizar el gráfico con el estado actual y obtiene el estado.
    state_log = actualize_graph_carga(conn)

    # Si no hubo errores, actualiza el mensaje de log para indicar éxito.
    if log_list == " ":
        log_list = "All operations were successful."

    print("Número de operaciones realizadas:", operation_count)
    print("Número total de palets transferidos:", palets_number)
    print("Costo total asociado a los palets transferidos:", palets_cost)
    print("Conjunto de nodos de origen únicos:", unique_origin_nodes)
    print("Conjunto de nodos de destino únicos:", unique_destination_nodes)
    print("Registro de errores o mensajes importantes:", log_list)
    print("Lista de nodos actualizados:", nodes_batch_full)
    print("Lista de aristas actualizadas:", edges_batch_full)


    # Devuelve un conjunto de resultados incluyendo logs, un botón para recargar el mapa y KPIs actualizados (elementos gradio)
    return (
        log_list,
        gr.Button("Recargar mapa", elem_classes="otherbutton", visible=True),
        gr.Number(label="KPI : Numero de operaciones", value=operation_count, visible=True, elem_classes="dropdown"), 
        gr.Number(label="KPI : Numero de palets", value=palets_number, visible=True, elem_classes="dropdown"), 
        gr.Number(label="KPI : Numero de origen", value=len(unique_origin_nodes), visible=True, elem_classes="dropdown"),  
        gr.Number(label="KPI : Destinaciones diferentes", value=len(unique_destination_nodes), visible=True, elem_classes="dropdown"),
        gr.Number(label="KPI : Coste total", value=palets_cost, visible=True, elem_classes="dropdown"), 
        gr.Number(label="KPI : Coste por palet", value=round(palets_cost / 1, 1), visible=True, elem_classes="dropdown") 
    )


# log_messages = make_daily_movements(r"C:\Users\JulienRigot\OneDrive - LIS Data Solutions\Escritorio\code_GORDIAS\base de datos graph\Tigergraph\CSV_entrega\df_1_move.csv", "CODE_Origin", "CODE_Destination", "Palets")

SyntaxError: parameter without a default follows parameter with a default (4240550438.py, line 332)

# Funciones de modificacion de red

Aqui siempre utilizo conn1 y no conn porque se modificaba solo el segundo grafo, creo que se deberia parametrizar "conn1"

In [5]:
import json
global logs_modif_network
logs_modif_network = " "



# ADD NODES 

def add_node(id, attributes_dict=None, vertex_type="Nodes"):
    global logs_modif_network
    if attributes_dict is None:
        conn1.upsertVertex(vertex_type, id)
        message =  f"Nodo {id} añadido\n"
        logs_modif_network +=message
        return logs_modif_network
    else:
        attributes = json.loads(attributes_dict)
        conn1.upsertVertex(vertex_type, id, attributes=attributes)
        message =  f"Nodo {id} añadido with attributos : {attributes} \n"
        logs_modif_network +=message
    return logs_modif_network

def get_values_nodes(*args):
    values = {key: val for key, val in zip(atributes_nodes.keys(), args)}
    return values

def get_values_edges(*args):
    values = {key: val for key, val in zip(atributes_edges.keys(), args)}
    return values["Capacity"]


def add_node_with_values(id, *args):
    attributes = get_values_nodes(*args)
    attributes_json = json.dumps(attributes)
    actualize_graph_carga(conn)
    return add_node(id, attributes_json)



# ADD EDGES 
def add_new_edge(source_vertex_id, target_vertex_id, attributes_dict=None,source_vertex_type="Nodes",target_vertex_type = "Nodes", edge_type="distribute_to"):
    global logs_modif_network
    if attributes_dict is None:
        conn1.upsertEdge(source_vertex_type, source_vertex_id, edge_type, target_vertex_type, target_vertex_id)
        actualize_graph_carga(conn)
        message =  f"Arista de {source_vertex_id} hasta {target_vertex_id}  añadida"
        logs_modif_network += message
        return logs_modif_network
    else:
        attributes = json.loads(attributes_dict)
        actualize_graph_carga(conn)
        conn1.upsertEdge(source_vertex_type, source_vertex_id, edge_type, target_vertex_type, target_vertex_id, attributes=attributes)
        message =  f"Arista de {source_vertex_id} hasta {target_vertex_id} añadida con los atributos siguientes: {attributes} \n"
        logs_modif_network += message
    return logs_modif_network

def add_edge_with_values(source_vertex_id, target_vertex_id, *args):
    attributes = get_values_edges(*args)
    attributes_json = json.dumps(attributes)
    return add_new_edge(source_vertex_id,target_vertex_id, attributes_json)

# GET ATTRIBUTES
attributes_nodes_list = []
attributes_edges_list = []
conex_nodes_list = conn1.getVertexAttrs('Nodes')
conex_edges_list = conn1.getEdgeAttrs('distribute_to')

for node in conex_nodes_list:
    attributes_nodes_list.append(node[0])


for edge in conex_edges_list:
    attributes_edges_list.append(edge[0])


#Remove carga atributes as it is here only for a cheat with colors of the tigergraph map. no need to change that
if 'Carga' in attributes_nodes_list:
    attributes_nodes_list.remove('Carga')

if 'Carga' in attributes_edges_list:
    attributes_edges_list.remove('Carga')


# GET NODES NAMES
list_name_nodes = []
name_nodes_conn = conn1.getVertices("Nodes")
for i in range(0, len(name_nodes_conn)):
    list_name_nodes.append(name_nodes_conn[i]["v_id"])

def add_edge(source_vertex_id, target_vertex_id, attributes_dict=None, vertex_type="Nodes", edgeType="distribute_to"):
    global logs_modif_network
    if attributes_dict is None:
        conn1.upsertEdge(sourceVertexType=vertex_type,
                        targetVertexType=vertex_type,
                        edgeType=edgeType,
                        sourceVertexId=source_vertex_id,
                        targetVertexId=target_vertex_id)
        message =  f"Arista desde {source_vertex_id} hasta {target_vertex_id} añadido\n"
        logs_modif_network +=message
        return logs_modif_network
    else:
        attributes = json.loads(attributes_dict)
        conn1.upsertEdge(sourceVertexType=vertex_type,
                        targetVertexType=vertex_type,
                        edgeType=edgeType,
                        sourceVertexId=source_vertex_id,
                        targetVertexId=target_vertex_id,
                        attributes=attributes)
        message =  f"Arista {source_vertex_id} hasta {target_vertex_id} añadido con los atributos siguientes : {attributes} \n"
        logs_modif_network +=message
        return logs_modif_network


#Erase Data

def erase_edge(source_vertex_id, target_vertex_id, vertex_type="Nodes", edgeType="distribute_to"):
    global logs_modif_network
    result = conn1.delEdges(sourceVertexType=vertex_type,
                           targetVertexType=vertex_type,
                           edgeType=edgeType,
                           sourceVertexId=source_vertex_id,
                           targetVertexId=target_vertex_id)
    
    if result == {edgeType: 0}:
        message = f"Arista de {source_vertex_id} hacia {target_vertex_id} no existe o no ha podido borrarse\n"
    else:
        message = f"Arista de  {source_vertex_id} hacia {target_vertex_id} borrada \n"
    
    logs_modif_network += message
    return logs_modif_network

    


def erase_node(id, vertex_type="Nodes"):
    global logs_modif_network
    result = conn1.delVerticesById(vertexIds=id, vertexType=vertex_type)
    
    if result == 0:
        message = f"Nodo {id} no existe o no ha podido borrarse\n"
    else:
        message = f"Nodo {id} borrado\n"
    
    logs_modif_network += message
    return logs_modif_network




# add_edge_with_values("Las Palmas de Gran Canaria","Seville",{"Capacity": 0, "Daily_movement": 0, "Price": 0})



In [15]:
atributes_nodes = {}
atributes_edges = {}
attributes_nodes_list = []
attributes_edges_list = []
conex_nodes_list = conn1.getVertexAttrs('Nodes')
conex_edges_list = conn1.getEdgeAttrs('distribute_to')

for node in conex_nodes_list:
    attributes_nodes_list.append(node[0])

atributes_nodes = {key: None for key in attributes_nodes_list}

for edge in conex_edges_list:
    attributes_edges_list.append(edge[0])
atributes_edges = {key: None for key in attributes_edges_list}


def add_node_with_values(id, *args):
    attributes = get_values_nodes(*args)
    attributes_json = json.dumps(attributes)
    return add_node(id, attributes_json)

node_id = "TEST"
attributes_to_update = {
    "LoadCapacity": 99
}

# Convert the attributes dictionary to a JSON string
attributes_json = json.dumps(attributes_to_update)

# Call the add_node function to upsert the node with the new attributes
logs = add_node(node_id, attributes_json)

# Print the logs to see the output
print(logs)

test = add_node_with_values(node_id,attributes_to_update)

TigerGraphException: ('Processing attribute LoadCapacity failed, value cannot be converted to FLOAT', 'REST-30200')

In [16]:
# Define the node ID and the new attribute you want to set


 Nodo TEST añadido with attributos : {} 
Nodo TEST añadido with attributos : {} 
Nodo TEST añadido with attributos : {'LoadCapacity': 99} 



# Tab modif cargas

In [ ]:
def adjust_charge(conn, percentage, operation="increase"):
    """
    Ajusta el movimiento diario de todas las aristas del tipo 'distribute_to' en un porcentaje especificado.
    Puede incrementar o decrementar el movimiento diario según la operación indicada.

    Parámetros:
    - conn: Conexión a la base de datos de TigerGraph.
    - percentage: Porcentaje por el cual ajustar el movimiento diario de las aristas.
    - operation: Tipo de ajuste a realizar ('increase' para incrementar, 'decrease' para decrementar).

    Retorna:
    - results: Diccionario con registros de logs y, si operation es 'decrease', conteo de aristas por nivel de carga.
    - new_charges: Lista de tuplas con las aristas actualizadas y sus nuevos movimientos diarios.
    """
    # Obtiene todas las aristas del tipo 'distribute_to'.
    aristas = conn.getEdgesByType(edgeType="distribute_to")
    new_charges = []
    
    # Variables para logs
    logs_state_aristas = ""
    
    # Variables para 'decrease' operation
    arista_counts = {"Sobrecargado": 0, "Subcargado": 0, "Optimal": 0} if operation == "decrease" else None
    
    for arista in aristas:
        current_charge = arista["attributes"]["Daily_movement"]  # Movimiento diario actual.
        capacity_edge = arista["attributes"]["Capacity"]  # Capacidad de la arista.

        if operation == "increase":
            # Calcula el nuevo movimiento diario incrementándolo.
            new_charge = current_charge * (1 + percentage / 100)
        elif operation == "decrease":
            # Calcula el nuevo movimiento diario decrementándolo.
            new_charge = current_charge * (1 - percentage / 100)
        else:
            raise ValueError("La operación debe ser 'increase' o 'decrease'.")

        # Actualiza el atributo en la arista.
        arista["attributes"]["Daily_movement"] = new_charge

        if operation == "decrease":
            # Determina el nivel de carga de la arista después del ajuste.
            if capacity_edge == 0:
                nivel_arista = "No definido"
            else:
                ratio = new_charge / capacity_edge
                if ratio > 1:
                    nivel_arista = "Sobrecargado"
                    arista_counts["Sobrecargado"] += 1
                    logs_state_aristas += (
                        f"⚠️ La arista {arista['from_id']} hasta {arista['to_id']} es "
                        f"{nivel_arista} con un movimiento diario de {new_charge}\n"
                    )
                elif 0.7 < ratio <= 1:
                    nivel_arista = "Optimal"
                    arista_counts["Optimal"] += 1
                else:
                    nivel_arista = "Subcargado"
                    arista_counts["Subcargado"] += 1
            
            # Agrega la arista actualizada con su nuevo movimiento diario y nivel de carga a la lista.
            new_charges.append((
                arista["from_id"],
                arista["to_id"],
                {"Daily_movement": new_charge, "Carga": nivel_arista}
            ))
        else:
            # Para 'increase', solo actualiza el movimiento diario.
            new_charges.append((
                arista["from_id"],
                arista["to_id"],
                {"Daily_movement": new_charge}
            ))
            # Añade un log para cada incremento.
            logs_state_aristas += (
                f"✅ La arista {arista['from_id']} hasta {arista['to_id']} ha sido incrementada a {new_charge}\n"
            )
    
    if operation == "decrease":
        # Si no se registraron aristas sobrecargadas, se añade un mensaje indicando que ninguna lo está.
        if logs_state_aristas == "":
            logs_state_aristas = "Ninguna arista esta sobrecargado.\n"
        
        # Crea un diccionario con los logs y el conteo de aristas por nivel de carga.
        results = {
            "logs": logs_state_aristas,
            "arista_counts": arista_counts,
        }
    else:
        # Para 'increase', crea un diccionario solo con los logs.
        results = {
            "logs": logs_state_aristas
        }
    
    return results, new_charges

def update_batch_adjust(conn, percentage, operation="increase"):
    """
    Actualiza en lote las aristas incrementando o decrementando su movimiento diario en un porcentaje especificado.

    Parámetros:
    - conn: Conexión a la base de datos de TigerGraph.
    - percentage: Porcentaje por el cual ajustar el movimiento diario de las aristas.
    - operation: Tipo de ajuste a realizar ('increase' para incrementar, 'decrease' para decrementar).

    Retorna:
    - Si operation es 'increase':
        - results: Diccionario con registros de logs.
    - Si operation es 'decrease':
        - results: Diccionario con registros de logs y conteo de aristas por nivel de carga.
    """
    if operation not in ["increase", "decrease"]:
        raise ValueError("La operación debe ser 'increase' o 'decrease'.")

    # Obtiene los logs y las aristas actualizadas con el ajuste aplicado.
    results, batch = adjust_charge(conn, percentage, operation=operation)
    
    # Actualiza las aristas en la base de datos con los nuevos movimientos diarios.
    conn.upsertEdges(
        sourceVertexType="Nodes",
        targetVertexType="Nodes",
        edgeType="distribute_to",
        edges=batch
    )
    
    return results



# test = update_batch_adjust(conn,10,"decrase")

In [39]:
def read_csv_path(input_file):
    df = pd.read_csv(input_file.name)
    target_column = sorted(list(df.columns))
    return  gr.Dropdown(choices = target_column) , gr.Dropdown(choices = target_column) , gr.Dropdown(choices = target_column)

In [40]:
name_edge = "distribute_to"
name_node = "Nodes"


#######RED ORIGINAL######
#Hay que poner en funccion aqui. 
numEdges = conn.getEdgeCount(name_edge)
numNodes = conn.getVertexCount(name_node)
edge = conn.getEdgeStats(name_edge)
avg_capacity = round(edge[name_edge]['Capacity']['AVG'],1)

edges = conn.getEdgesByType(name_edge)
total_capacity_edges = 0
for warehouse in edges :
    capacity_edges = warehouse["attributes"]["Capacity"]     
    total_capacity_edges += round(capacity_edges,1)

almacenes = conn.getVertices(name_node)
total_capacity_warehouse = 0
for warehouse in almacenes :
    capacity_edges = warehouse["attributes"]["Capacity"]      
    total_capacity_warehouse += round(capacity_edges,1)
    total_capacity_warehouse = round(total_capacity_warehouse,1)

almacen = conn.getVertices(name_node)
total_stock = 0
for warehouse in almacen :
    stock = warehouse["attributes"]["Stock"]     
    total_stock += stock
    total_stock = round(total_stock,0)

capacity_per_edge = round(total_capacity_warehouse/numEdges,1)


#######RED SIMULADA######
import concurrent.futures

def get_num_edges(conn, edge_name):
    return conn.getEdgeCount(edge_name)

def get_num_nodes(conn, node_name):
    return conn.getVertexCount(node_name)

def get_avg_capacity(conn, edge_name):
    edges = conn.getEdgeStats(edge_name)
    avg_capacity = edges[edge_name]['Capacity']['AVG']
    return round(avg_capacity/2, 1)

def get_total_capacity_edges(conn, edge_name):
    edges = conn.getEdgesByType(edge_name)
    total_capacity = 0
    for edge in edges:
        total_capacity += edge["attributes"]["Capacity"]
    return total_capacity /2

def get_total_capacity_nodes(conn, node_name):
    warehouses = conn.getVertices(node_name)
    total_capacity = 0
    for warehouse in warehouses:
        total_capacity += warehouse["attributes"]["Capacity"]
    return round(total_capacity, 1)

def get_total_stock(conn, node_name):
    warehouses = conn.getVertices(node_name)
    total_stock = 0
    for warehouse in warehouses:
        total_stock += warehouse["attributes"]["Stock"]
    return round(total_stock, 0)

def get_capacity_per_edge(total_capacity_warehouses, num_edges):
    if num_edges == 0:
        return 0
    return round(total_capacity_warehouses / num_edges, 0)

def get_KPI(conn, edge_name, node_name):
     with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = {
            "num_edges": executor.submit(get_num_edges, conn, edge_name),
            "num_nodes": executor.submit(get_num_nodes, conn, node_name),
            "avg_capacity": executor.submit(get_avg_capacity, conn, edge_name),
            "total_capacity_edges": executor.submit(get_total_capacity_edges, conn, edge_name),
            "total_capacity_warehouses": executor.submit(get_total_capacity_nodes, conn, node_name),
            "total_stock": executor.submit(get_total_stock, conn, node_name)
        }

        results = {name: future.result() for name, future in futures.items()}
        results["capacity_per_edge"] = get_capacity_per_edge(results["total_capacity_warehouses"], results["num_edges"])
        
        return results

# Instalar Algoritmos A*

In [ ]:
#USE GRAPH VWG = Nombre del grafo (se necesita instalar por cada grafo y no cada cluster ! )
#Create Query "Nombre de la query"

result = conn.gsql("""
USE GRAPH VWG
CREATE QUERY tg_astar (VERTEX source_vertex, VERTEX target_vertex, SET<STRING> e_type_set,
STRING weight_type, STRING latitude, STRING longitude,
STRING weight_attribute, BOOL print_stats = False) SYNTAX V1 {

TYPEDEF TUPLE<FLOAT dist, VERTEX v> pathTuple;    # <shotest distance, parent node>
HeapAccum<pathTuple>(1, dist ASC) @@find_min_v_heap;  # retain 1 shortest path
HeapAccum<pathTuple>(1, dist ASC) @min_dist_heap;
OrAccum @or_visited, @@or_valid_path_exists;
ListAccum<VERTEX> @@tmp_list;  # the optimal node
SumAccum<FLOAT> @@sum_total_dist;  # the shortest distance
SetAccum<EDGE> @@display_edge_set;
SetAccum<VERTEX> @@display_node_set;
INT hop;
FLOAT x1,y1;

# Check weight_type parameter
IF weight_type NOT IN ("INT", "FLOAT", "DOUBLE") THEN
    PRINT "weight_type must be INT, FLOAT, or DOUBLE" AS errMsg;
    RETURN;
END;

# record target latitude and longitude
Tgt = {target_vertex};
Tgt = SELECT s
      FROM Tgt:s
      POST-ACCUM x1 = s.getAttr(latitude,"FLOAT"),
	         y1 = s.getAttr(longitude,"FLOAT");

Start = {source_vertex};   # the optimal node
Opt = {source_vertex};    # all of the optimal nodes

Start = SELECT s
        FROM Start:s
        ACCUM s.@or_visited = True,
              s.@min_dist_heap = pathTuple(0,s);

# run aster to find shortest distance greedily
WHILE Opt.size() > 0 DO
    # find the node with shortest distance
    Opt = SELECT t
          FROM Opt:s-(e_type_set:e)-> :t
          WHERE t.@or_visited == False
	  ACCUM
              # we use Haversine formula as the heuristic function here
              CASE weight_type WHEN "INT" THEN
                  t.@min_dist_heap += pathTuple(s.@min_dist_heap.top().dist + e.getAttr(weight_attribute, "INT")
		  +  tg_GetDistance(t.getAttr(latitude,"FLOAT"),t.getAttr(longitude,"FLOAT"),x1,y1),s)
              WHEN "FLOAT" THEN
                  t.@min_dist_heap += pathTuple(s.@min_dist_heap.top().dist + e.getAttr(weight_attribute, "FLOAT")
		  +  tg_GetDistance(t.getAttr(latitude,"FLOAT"),t.getAttr(longitude,"FLOAT"),x1,y1),s)
              WHEN "DOUBLE" THEN
                  t.@min_dist_heap += pathTuple(s.@min_dist_heap.top().dist + e.getAttr(weight_attribute, "DOUBLE")
		  +  tg_GetDistance(t.getAttr(latitude,"FLOAT"),t.getAttr(longitude,"FLOAT"),x1,y1),s)
              END;

    Opt = SELECT t
          FROM Start:s-(e_type_set:e)-> :t
          WHERE t.@or_visited == False
          POST-ACCUM @@find_min_v_heap += pathTuple(t.@min_dist_heap.top().dist,t);

    @@tmp_list.clear();
    IF @@find_min_v_heap.size() > 0 THEN
        @@tmp_list += @@find_min_v_heap.pop().v;
    END;

    Opt = {@@tmp_list};
    Start = Opt UNION Start;
    Opt = SELECT t
          FROM Opt:t
	  POST-ACCUM
	      t.@or_visited += True;
              # Determine if it is the target point and terminate the loop if it is
              IF @@tmp_list.get(0) == target_vertex THEN
                  BREAK;
              END;

END;
# The test is whether there is a path between two points
Start = {target_vertex};
Start = SELECT s
        FROM Start:s
        POST-ACCUM @@or_valid_path_exists += s.@min_dist_heap.size() > 0,
                   @@display_node_set += s;

IF @@or_valid_path_exists THEN
    # find path
    WHILE Start.size() > 0 DO
        Start =
	SELECT t
	FROM Start:s-(e_type_set:e)-> :t
        WHERE t == s.@min_dist_heap.top().v
        ACCUM
	    @@display_edge_set += e,
            CASE weight_type WHEN "INT" THEN
                @@sum_total_dist += e.getAttr(weight_attribute, "INT")
            WHEN "FLOAT" THEN
                @@sum_total_dist += e.getAttr(weight_attribute, "FLOAT")
            WHEN "DOUBLE" THEN
                @@sum_total_dist += e.getAttr(weight_attribute, "DOUBLE")
            END,
            @@display_node_set += t;
            hop = hop + 1;
    END;
    hop = hop - 1;
    PRINT @@sum_total_dist;
    PRINT hop;
    IF print_stats THEN
        tmp = {@@display_node_set};
        PRINT @@display_edge_set,tmp;
    END;
ELSE
    PRINT "No viable path found.";
END;
}""")

Una vez definida la query, se necesita instalar con : 

In [ ]:
result = conn.gsql("""
USE GRAPH VWG           #Nombre del grafo
INSTALL QUERY tg_astar  #Nombre del algoritmo
""")

Deberias tener un resultado del estilo :
 
Using graph \'VWG\'\nStart installing queries, about 1 minute ...\ntg_astar query: curl -X GET \'https://127.0.0.1:9000/query/VWG/tg_astar?source_vertex=VALUE&source_vertex.type=VERTEX_TYPE&target_vertex=VALUE&target_vertex.type=VERTEX_TYPE&e_type_set=VALUE&weight_type=VALUE&latitude=VALUE&longitude=VALUE&weight_attribute=VALUE&[print_stats=VALUE]\'